In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import norm
from statsmodels.sandbox.regression.gmm import GMM
from statsmodels.base.model import GenericLikelihoodModel

In [2]:
data = pd.read_csv('data.csv')

#pre-processing to calculate outside good shares
shares = data[['Market_ID','Inside Good Share','PPO']].copy()
shares['PPO Share'] = data['Inside Good Share'] * data['PPO']
shares['HMO Share'] = data['Inside Good Share'] * (1 - data['PPO'])

group_shares = shares.groupby('Market_ID').sum()
group_shares['Outside Good Share'] = 1 - group_shares['Inside Good Share']

data = pd.merge(data,group_shares[['Outside Good Share']], 
                right_index=True, left_on = 'Market_ID')

print data.min()

Market_ID             1.000000
Plan_ID               1.000000
PPO                   0.000000
Network Score         0.780000
Satisfaction Score    0.725000
Premium               2.269728
Inside Good Share     0.037670
Outside Good Share    0.096070
dtype: float64


In [3]:
#first estimate using logit
class logit(GMM):
    
    def __init__(self, *args, **kwds):
        # set appropriate counts for moment conditions and parameters
        super(logit, self).__init__(*args, **kwds)

        
    def momcond(self, params):
        #unwrap stuff
        shares = np.array(self.endog).transpose()
        exog = np.array(self.exog)
        instr = np.array(self.instrument)
        
        lshare = np.log(shares[0]) -  np.log(shares[1])
        lshare = lshare.transpose()
       
        lshare_fit = np.matmul(exog,params) #linear equation    
        
        xi = lshare_fit - lshare
        g = instr * xi[:, np.newaxis]
        
        return g

In [4]:
#calculate hausmann insturments
mkt_dum = pd.get_dummies(data['Market_ID'],prefix='mkt',drop_first=True)
plan_dum = pd.get_dummies(data['Plan_ID'],prefix='plan',drop_first=True)
hausman_instr = plan_dum

#set up x and y
y = data[['Inside Good Share','Outside Good Share']]
x =  data[['Network Score','Satisfaction Score','PPO','Premium']]

In [5]:
#set up initial est
beta_init = np.full(len(x.columns),1)

#set up model
model = logit(y , x, hausman_instr)

result = model.fit(beta_init, maxiter=2, optim_method='nm', wargs=dict(centered=False))
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.000008
         Iterations: 292
         Function evaluations: 497
Optimization terminated successfully.
         Current function value: 0.005149
         Iterations: 139
         Function evaluations: 249
                                             logit Results                                             
Dep. Variable:     ['Inside Good Share', 'Outside Good Share']   Hansen J:                        16.99
Model:                                                   logit   Prob (Hansen J):                 0.108
Method:                                                    GMM                                         
Date:                                         Tue, 09 Oct 2018                                         
Time:                                                 17:09:37                                         
No. Observations:                                         3300                                  

In [6]:
#compute exp(delta_j)
delta = data.copy()[['Plan_ID','Market_ID','PPO','Premium']]
x = np.array(x)
beta = result.params
alpha = abs(beta[3])
delta['exp_delta'] = np.exp(np.matmul(x,beta))

#compute 1 + sum_j exp(delta_j)
sum_delta = delta.groupby('Market_ID').sum()
sum_delta['sum_exp_delta'] = 1 + sum_delta['exp_delta'] 

delta = pd.merge(delta, sum_delta[['sum_exp_delta']], 
                right_index=True, left_on = 'Market_ID')

#compute s_j
delta['fitted_share'] = delta['exp_delta']/delta['sum_exp_delta']

print delta.groupby('Market_ID').sum().mean()

print delta['fitted_share'].max(), delta['fitted_share'].min(), delta['fitted_share'].mean(), np.sqrt(delta['fitted_share'].var())

Plan_ID          47.138333
PPO               2.720000
Premium          13.562795
exp_delta         6.297526
sum_exp_delta    44.228315
fitted_share      0.843098
dtype: float64
0.3211705667781474 0.04175067681732296 0.153290506773 0.0527060652944428


In [7]:
#agregate elasticities
delta = delta.groupby('Plan_ID').mean()

shares = np.array(delta['fitted_share'])
prices = np.array(delta['Premium'])

#set up matrix skeleton
own_price = np.identity(len(shares))
cross_price = 1 - own_price

#actually calculate elasticity
cross_elasticity = shares * alpha * prices
own_elasticity  = -(1-shares) * alpha * prices

elasticity =  cross_price*cross_elasticity + own_price *own_elasticity

print elasticity[1][1],elasticity[2][1],elasticity[3][1] #second index tells you wrt what good i.e. denom

-4.308596601690402 0.5554552982100271 0.5554552982100271


In [8]:
#solve for marginal costs
p_divide_q = (1/shares * np.ones( (len(shares), len(shares)) )).transpose() * prices

print p_divide_q[0][1], prices[1]/shares[0]

inv_derivative = np.linalg.inv( elasticity * p_divide_q)

mc = np.array([prices]).transpose() - np.matmul(inv_derivative, np.array([shares]).transpose()) 

mc = mc.transpose()[0]

print mc

12.410684346595144 12.410684346595144
[2.59083602 2.35673823 2.60768873 2.52299844 2.33739461 2.35360695
 2.59488989 2.40818186 2.52999745 2.33440917 2.32182747 2.59993908
 2.51966688 2.40866557 2.59619547 2.36380635]


In [9]:
#simplified version to get things working

def profits(p):
    p[p < 0] = 0 #set things if necessary?
    xes =  data[['Plan_ID','Network Score','Satisfaction Score','PPO']].groupby('Plan_ID').mean()
    constant = np.exp(np.matmul(xes,beta[:-1]))
    delta_js = constant*np.exp(alpha*p)
    denom = 1+sum(delta_js)
    s_js = delta_js/denom
    return - np.matmul(s_js, np.array([prices-mc+.25]).transpose())

prices = np.array(delta['Premium'])

sum(profits(prices))

-0.2511224321308708

In [10]:
import numpy as np
from scipy.optimize import minimize

res = minimize(profits, prices, method='nelder-mead',
     options={'xtol': 1e-8, 'disp': True})

print(res.x) #doesn't seem right

Optimization terminated successfully.
         Current function value: -0.252226
         Iterations: 285
         Function evaluations: 853
[1.00271692e-02 9.75547188e-02 1.11861770e-01 4.75887662e+00
 1.24817262e+01 7.04289160e+00 3.91753341e+00 8.20092864e+00
 9.78134182e+00 1.68702280e+01 3.86840732e-01 3.28303747e+00
 2.09218577e+01 3.69849983e+01 4.98921183e-02 1.37640899e+01]


In [ ]:
#profits per enrollee
#outside good shares
#consumer surplus